In [ ]:
'''
Develop risk models using survival data and a combination of linear and non-linear techniques.
We'll be using a dataset with survival data of patients with Primary Biliary Cirrhosis (pbc). 
PBC is a progressive disease of the liver caused by a buildup of bile within the liver (cholestasis)
that results in damage to the small bile ducts that drain bile from the liver.

In [ ]:

    Cox Proportional Hazards
        Data Preprocessing for Cox Models.
    Random Survival Forests
        Permutation Methods for Interpretation.


In [ ]:
!pip install lifelines 

In [ ]:
import numpy as  np 
import pandas as pd
import matplotlib.pyplot as plt 
from util import load_data 
from sklearn.model_selection  import train_test_split
from lifelines import CoxPHFitter 
from lifelines.utils import concordance_index as cindex

In [ ]:
df = load_data()
df.head()

,time,status,trt,age,sex,ascites,hepato,spiders,edema,bili,chol,albumin,copper,alk.phos,ast,trig,platelet,protime,stage
0,1.095890,1.0,0.0,58.765229,0.0,1.0,1.0,1.0,1.0,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4.0
1,12.328767,0.0,0.0,56.446270,0.0,0.0,1.0,1.0,0.0,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3.0
2,2.772603,1.0,0.0,70.072553,1.0,0.0,0.0,0.0,0.5,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4.0
3,5.273973,1.0,0.0,54.740589,0.0,0.0,1.0,1.0,0.5,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4.0
6,5.019178,0.0,1.0,55.534565,0.0,0.0,1.0,0.0,0.0,1.0,322.0,4.09,52.0,824.0,60.45,213.0,204.0,9.7,3.0


In [ ]:
df_dev,df_test = train_test_split(df,test_size=0.2)
df_train,df_val = train_test_split(df_dev,test_size=0.25)
print(f"total number of patients: {df.shape[0]}")
print(f"total number of patients in training set: {df_train.shape[0]}")
print(f"total number of patients in validation set: {df_val.shape[0]}")
print(f"total number of patients in test set: {df_test.shape[0]}")

total number of patients: 258
total number of patients in training set: 154
total number of patients in validation set: 52
total number of patients in test set: 52


In [ ]:
continuous_columns = ['age', 'bili', 'chol', 'albumin', 'copper', 'alk.phos', 'ast', 'trig', 'platelet', 'protime']
mean =  df_train.loc[:,continuous_columns].mean()
std = df_train.loc[:,continuous_columns].std()
df_train.loc[:,continuous_columns] = (df_train.loc[:,continuous_columns] - mean) / std
df_val.loc[:,continuous_columns] = (df_val.loc[:,continuous_columns] - mean) / std
df_test.loc[:,continuous_columns] = (df_test.loc[:,continuous_columns] - mean) / std

In [ ]:
df_train.loc[:,continuous_columns].describe()

,age,bili,chol,albumin,copper,alk.phos,ast,trig,platelet,protime
count,1.540000e+02,1.540000e+02,1.540000e+02,1.540000e+02,1.540000e+02,1.540000e+02,1.540000e+02,1.540000e+02,1.540000e+02,1.540000e+02
mean,-1.586033e-17,-2.667419e-17,1.586033e-17,4.037175e-17,2.847650e-17,-1.081386e-17,-4.902283e-17,-1.441848e-18,7.930164e-18,2.162772e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.936059e+00,-6.388796e-01,-1.098211e+00,-3.746299e+00,-1.258592e+00,-7.955839e-01,-1.496346e+00,-1.147684e+00,-2.033393e+00,-1.698186e+00
25%,-6.744601e-01,-5.403345e-01,-5.165212e-01,-4.084420e-01,-7.480490e-01,-4.952114e-01,-7.603799e-01,-5.397509e-01,-6.555874e-01,-7.988028e-01
50%,-3.603468e-02,-4.220803e-01,-2.618861e-01,6.133052e-02,-3.008841e-01,-3.079971e-01,-1.222797e-01,-2.621399e-01,-2.297619e-02,-1.992140e-01
75%,6.075176e-01,6.591006e-03,1.878135e-01,5.558279e-01,5.265471e-01,2.292820e-02,5.851602e-01,3.493069e-01,6.310794e-01,4.753234e-01
max,2.640969e+00,4.820521e+00,6.634287e+00,2.237119e+00,4.036968e+00,5.224340e+00,3.702372e+00,6.639479e+00,2.855941e+00,4.397633e+00


In [ ]:
def to_one_hot(dataframe,columns):
  one_hot_df = pd.get_dummies(dataframe,columns=columns,drop_first=True,dtype=np.float64)
  return one_hot_df

In [ ]:
to_encode = ["edema","stage"]
one_hot_train = to_one_hot(df_train,to_encode)
one_hot_val = to_one_hot(df_val,to_encode)
one_hot_test = to_one_hot(df_test,to_encode)
print(one_hot_val.columns.tolist())
print(f"there are {len(one_hot_val.columns)} column")

['time', 'status', 'trt', 'age', 'sex', 'ascites', 'hepato', 'spiders', 'bili', 'chol', 'albumin', 'copper', 'alk.phos', 'ast', 'trig', 'platelet', 'protime', 'edema_0.5', 'edema_1.0', 'stage_2.0', 'stage_3.0', 'stage_4.0']
there are 22 column


In [ ]:
cph = CoxPHFitter()
cph.fit(one_hot_train,duration_col='time',event_col="status",step_size=0.1)

In [ ]:
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 154 total observations, 93 right-censored observations>
             duration col = 'time'
                event col = 'status'
      baseline estimation = breslow
   number of observations = 154
number of events observed = 61
   partial log-likelihood = -216.67
         time fit was run = 2021-05-30 16:11:33 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
trt         0.09       1.09       0.33            -0.57             0.74                 0.57                 2.10
age         0.33       1.39       0.20            -0.06             0.71                 0.95                 2.04
sex         0.37       1.45       0.51            -0.63             1.38                 0.53                 3.96
ascites    -0.18       0.84       0.63            -1.41             1.06                 0.24                 2.88
hepato      0.27       1.31       0.36            -0.44             0.98                 0.64                 2.65
spiders    -0.41       0.66       0.41            -1.21             0.38                 0.30                 1.47
bili        0.55       1.74       0.19             0.18             0.92                 1.20                 2.52
chol        0.08       1.08       0.15            -0.22             0.38                 0.80                 1.46
albumin    -0.59       0.56       0.18            -0.95            -0.22                 0.39                 0.80
copper      0.21       1.24       0.18            -0.14             0.56                 0.87                 1.76
alk.phos   -0.18       0.84       0.13            -0.44             0.08                 0.64                 1.09
ast         0.37       1.45       0.18             0.03             0.72                 1.03                 2.05
trig       -0.02       0.98       0.13            -0.28             0.24                 0.76                 1.27
platelet    0.07       1.08       0.16            -0.25             0.39                 0.78                 1.48
protime     0.33       1.40       0.17             0.00             0.66                 1.01                 1.94
edema_0.5   0.24       1.27       0.52            -0.78             1.25                 0.46                 3.50
edema_1.0   1.07       2.90       0.64            -0.19             2.32                 0.83                10.18
stage_2.0   1.00       2.71       1.11            -1.17             3.16                 0.31                23.66
stage_3.0   0.40       1.50       1.10            -1.76             2.57                 0.17                13.03
stage_4.0   0.49       1.63       1.18            -1.82             2.80                 0.16                16.43

              z      p   -log2(p)
covariate                        
trt        0.26   0.79       0.33
age        1.67   0.09       3.40
sex        0.73   0.47       1.10
ascites   -0.28   0.78       0.36
hepato     0.74   0.46       1.12
spiders   -1.02   0.31       1.69
bili       2.93 <0.005       8.21
chol       0.51   0.61       0.71
albumin   -3.17 <0.005       9.37
copper     1.18   0.24       2.07
alk.phos  -1.33   0.18       2.45
ast        2.10   0.04       4.82
trig      -0.15   0.88       0.18
platelet   0.45   0.65       0.62
protime    1.99   0.05       4.42
edema_0.5  0.46   0.65       0.63
edema_1.0  1.66   0.10       3.38
stage_2.0  0.90   0.37       1.45
stage_3.0  0.36   0.72       0.48
stage_4.0  0.42   0.68       0.56
---
Concordance = 0.86
Partial AIC = 473.34
log-likelihood ratio test = 101.85 on 20 df
-log2(p) of ll-ratio test = 40.63

In [ ]:
cph.plot_covariate_groups('trt', values=[0, 1]);

In [ ]:
def hazard_ratio(case_1,case_2,cox_params):
  hr = np.exp(cox_params.dot((case_1 - case_2).T))
  return hr

In [ ]:
i = 1
case_1 = one_hot_train.iloc[i, :].drop(['time', 'status'])
j = 5
case_2 = one_hot_train.iloc[j, :].drop(['time', 'status'])
print(hazard_ratio(case_1.values, case_2.values, cph.params_.values))

0.2882762826192309


In [ ]:
def harrell_c(y_true, scores, event):
    
    n = len(y_true)
    assert (len(scores) == n and len(event) == n)
    
    concordant = 0.0
    permissible = 0.0
    ties = 0.0
    
    result = 0.0
    for i in range(n):
        for j in range(i+1, n):
            if event[i] == 1 or event[j] == 1:
                if event[i] == 1 and event[j] == 1:
                    permissible += 1.0
                    if scores[i] == scores[j]:
                        ties += 1.0
                    elif y_true[i] < y_true[j] and scores[i] > scores[j]:
                        concordant += 1.0
                    elif y_true[i] > y_true[j] and scores[i] < scores[j]:
                        concordant += 1.0
                elif event[i] != event[j]:
                    censored = j
                    uncensored = i
                    
                    if event[i] == 0:
                        censored = i
                        uncensored = j
                    if y_true[uncensored] <= y_true[censored]:
                        permissible += 1.0
                        if scores[uncensored] == scores[censored]:
                            ties += 1.0
                        if scores[uncensored] > scores[censored]:
                            concordant += 1.0
    result = (concordant + 0.5*ties) / permissible
    return result   

In [ ]:
y_true = [30, 12, 84, 9]
event = [1, 1, 1, 1]
scores = [0.5, 0.9, 0.1, 1.0]
print("Output: {}".format(harrell_c(y_true, scores, event)))

Output: 1.0


In [ ]:
scores = cph.predict_partial_hazard(one_hot_train)
cox_train_scores = harrell_c(one_hot_train['time'].values, scores.values, one_hot_train['status'].values)
scores = cph.predict_partial_hazard(one_hot_val)
cox_val_scores = harrell_c(one_hot_val['time'].values, scores.values, one_hot_val['status'].values)
scores = cph.predict_partial_hazard(one_hot_test)
cox_test_scores = harrell_c(one_hot_test['time'].values, scores.values, one_hot_test['status'].values)
print("Train:", cox_train_scores)
print("Val:", cox_val_scores)
print("Test:", cox_test_scores)

Train: 0.8581415174765559
Val: 0.7905092592592593
Test: 0.8118556701030928


In [ ]:
%load_ext rpy2.ipython
%R require(ggplot2)
from rpy2.robjects.packages import importr
base = importr('base')
utils = importr('utils')
import rpy2.robjects.packages as rpackages
forest = rpackages.importr('randomForestSRC', lib_loc='R')
from rpy2 import robjects as ro
R = ro.r
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [ ]:
model = forest.rfsrc(ro.Formula('Surv(time, status) ~ .'), data=df_train, ntree=300, nodedepth=5, seed=-1)

In [ ]:
print(model)

In [ ]:
result = R.predict(model, newdata=df_val)
scores = np.array(result.rx('predicted')[0])
print("Cox Model Validation Score:", cox_val_scores)
print("Survival Forest Validation Score:", harrell_c(df_val['time'].values, scores, df_val['status'].values))

In [ ]:
result = R.predict(model, newdata=df_test)
scores = np.array(result.rx('predicted')[0])
print("Cox Model Test Score:", cox_test_scores)
print("Survival Forest Validation Score:", harrell_c(df_test['time'].values, scores, df_test['status'].values))

In [ ]:
#just know that random surival forests come with their own built in variable importance feature.
#The method is referred to as VIMP

In [ ]:
vimps = np.array(forest.vimp(model).rx('importance')[0])

y = np.arange(len(vimps))
plt.barh(y, np.abs(vimps))
plt.yticks(y, df_train.drop(['time', 'status'], axis=1).columns)
plt.title("VIMP (absolute value)")
plt.show()